In [12]:
#%pip install pandas
#%pip install pymssql
#%pip install matplotlib
#%pip install sklearn

In [13]:
import pandas as pd
import pymssql
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np
from scipy.optimize import curve_fit

In [14]:
def ms_sql_con():
    sql_name = 'voice_ai'
    sql_server = '10.2.4.124'
    sql_login = 'ICECORP\\1c_sql'

    with open('sql.pass','r') as file:
        sql_pass = file.read().replace('\n', '')
        file.close()

    return pymssql.connect(
            server = sql_server,
            user = sql_login,
            password = sql_pass,
            database = sql_name,
        )

In [15]:
def read_sql(query):
    return pd.read_sql(query, con=ms_sql_con(), parse_dates=None)

In [ ]:
def phrases_per_hour(date_from, date_to, phrase):
    query = "select month(record_date) as m,  day(record_date) as d, DATEPART(HOUR, record_date) as h, count(id) as c" 
    query += " from transcribations where "
    query += " record_date > '"+date_from+"' and "
    query += " record_date < '"+date_to+"' and "
    query += " text like '%"+phrase+"%' "
    query += " group by month(record_date), day(record_date), DATEPART(HOUR, record_date)"
    query += " order by month(record_date), day(record_date), DATEPART(HOUR, record_date);"
    df = read_sql(query)
    
    df.drop(['m', 'd'], axis = 1, inplace = True)
    df = df.groupby(df.h).median()
    df.reset_index(level=0, inplace=True)
    
    plt.bar(df.h, df.c)
    plt.show()

In [ ]:
def silence_timing(date_from, date_to):

    def convert_false(val):
        return not val
    
    query = "select"
    query += " sum(end_time - start) as time,"
    query += " DATEPART(HOUR, record_date) as h,"
    query += " linkedid,"
    query += " side,"
    query += " duration"
    query += " from transcribations"
    query += " where"
    query += " record_date > '"+date_from+"' and "
    query += " record_date < '"+date_to+"' and"
    query += " duration > 0 "
    query += " group by record_date, linkedid, side, duration"
    query += " order by linkedid, side;"
    df = read_sql(query)
    df['a'] = df.time*df.side
    df.side = df.side.apply(convert_false)
    df['b'] = df.time*df.side
    df.drop(['time', 'side'], axis = 1, inplace = True)
    df = df.groupby(['h', 'linkedid', 'duration']).sum()
    df.reset_index(level=0, inplace=True)
    df.reset_index(level=0, inplace=True)
    df.reset_index(level=0, inplace=True)
    df.drop(['linkedid'], axis = 1, inplace = True)
    df.reset_index(level=0, inplace=True)
    df['part_a'] = df.a/df.duration
    df['part_b'] = df.b/df.duration
    df['part_silence'] = 1 - df.part_a - df.part_b
    df.drop(['duration', 'a', 'b'], axis = 1, inplace = True)
    df = df.groupby(['h']).mean()
    df.reset_index(level=0, inplace=True)
    df.drop(['index'], axis = 1, inplace = True)
    df.sort_values(['h'], inplace = True)
    df.plot(x = 'h', y = ['part_a', 'part_b', 'part_silence'], style = '.')    

In [ ]:
phrases_per_hour('2021-03-04 00:00:00', '2021-03-05 00:00:00', 'операторы заняты')

In [ ]:
silence_timing('2021-03-01 00:00:00', '2021-03-02 00:00:00')

In [ ]:
### developing

In [ ]:
query = "SELECT column_name FROM information_schema.columns WHERE table_name='transcribations';"
df = read_sql(query)
df

In [28]:
query = "SELECT audio_file_name, duration FROM transcribations where linkedid = '1621315463.3600548';"
df = read_sql(query)
df

,audio_file_name,duration


In [ ]:
query = "SELECT * FROM queue WHERE not file_size is null;"
df = read_sql(query)
df

In [17]:
query = "SELECT * FROM transcribations WHERE not file_size is null and file_size>0;"
df = read_sql(query)
df.head(2)

,transcribation_date,date_y,date_m,date_d,side,text,start,audio_file_name,conf,end_time,...,sentiment_neg,ID,linkedid,dst,record_date,source_id,src,cpu_id,duration,file_size
0,2021-05-18 16:18:52,None,None,None,False,ольга здравствуйте,8.16,in_9672079669_2021-05-18-13-17-26rxtx.wav,0.996855,9.21,...,0.0,23089484,1621333044.3611722,5024,2021-05-18 13:17:26,1,9672079669,12,185.8,5945644
1,2021-05-18 16:18:52,None,None,None,False,скажите пожалуйста завтра хотелось бы вызвать ...,10.35,in_9672079669_2021-05-18-13-17-26rxtx.wav,0.911732,15.81,...,0.0,23089498,1621333044.3611722,5024,2021-05-18 13:17:26,1,9672079669,12,185.8,5945644


In [19]:
df[df.source_id==1].transcribation_date.min()

Timestamp('2021-05-18 16:18:52')

In [20]:
df[df.source_id==2].transcribation_date.min()

Timestamp('2021-05-18 17:31:39')

In [22]:
query = "SELECT * FROM transcribations WHERE transcribation_date > '2021-05-18 17:31:39';"
df = read_sql(query)
df.head(2)

,transcribation_date,date_y,date_m,date_d,side,text,start,audio_file_name,conf,end_time,...,sentiment_neg,ID,linkedid,dst,record_date,source_id,src,cpu_id,duration,file_size
0,2021-05-18 17:31:40,None,None,None,True,и,0.061246,in_9153321111_2021-05-18-13-47-41rxtx.wav,1.0,1.98,...,0.0,23098446,1621334860.3612835,5028,2021-05-18 13:47:41,1,9153321111,3,247.04,7905324
1,2021-05-18 17:31:44,None,None,None,True,и,0.061262,in_9636780293_2021-05-18-13-58-49rxtx.wav,1.0,1.98,...,0.0,23098451,1621335528.3613267,5021,2021-05-18 13:58:49,1,9636780293,1,67.40,2156844


In [24]:
df[(df.duration==0) & (df.file_size>0)]

,transcribation_date,date_y,date_m,date_d,side,text,start,audio_file_name,conf,end_time,...,sentiment_neg,ID,linkedid,dst,record_date,source_id,src,cpu_id,duration,file_size
33563,2021-05-18 22:20:24,None,None,None,False,,0.0,in_9818210248_2021-05-18-16-45-00rxtx.wav,0.0,0.0,...,NaN,23132187,1621345498.3619125,None,2021-05-18 16:45:00,1,9818210248,9,0.0,44
33564,2021-05-18 22:20:24,None,None,None,True,,0.0,in_9818210248_2021-05-18-16-45-00rxtx.wav,0.0,0.0,...,NaN,23132188,1621345498.3619125,None,2021-05-18 16:45:00,1,9818210248,9,0.0,44
51870,2021-05-19 00:56:54,None,None,None,False,,0.0,a2021-05-18t18:40:17b_c9255710403d_e9773863405...,0.0,0.0,...,NaN,23150494,1621352415.502237,9773863405,2021-05-18 18:40:17,2,9255710403,6,0.0,44
51871,2021-05-19 00:56:54,None,None,None,False,,0.0,a2021-05-18t18:43:05b_c9255710403d_e4956112700...,0.0,0.0,...,NaN,23150495,1621352583.502312,4956112700,2021-05-18 18:43:05,2,9255710403,6,0.0,44


In [25]:
df.transcribation_date.max()

Timestamp('2021-05-19 07:33:05')

In [ ]:
# ===============

In [ ]:
query = "SELECT column_name FROM information_schema.columns WHERE table_name='perf_log';"
df = read_sql(query)
df

In [38]:
query = "select *"
query += " from perf_log "
#query += " where linkedid='1621315463.3600548';"
# 2021-05/18/in_9660679917_2021-05-18-08-24-28.wav49
query += " where event_date > '2021-05-18 00:00:00'"
query += " and event_date< '2021-05-18 16:00:00'"
query += " and file_name like '%9660679917%';"
df = read_sql(query)
df

,event_date,step,cpu,duration,linkedid,source_id,cores,time,file_name
0,2021-05-18 15:22:15,2,6,46.76,1621329796.484120,2,14,41.526574,a2021-05-18t12:23:18b_c9660679917d_e9254535117...
1,2021-05-18 15:22:15,0,6,46.76,1621329796.484120,2,14,42.304435,a2021-05-18t12:23:18b_c9660679917d_e9254535117...
2,2021-05-18 15:21:30,2,6,46.54,1621329796.484120,2,14,39.298516,a2021-05-18t12:23:18b_c9660679917d_e9254535117...
3,2021-05-18 15:21:30,0,6,46.54,1621329796.484120,2,14,39.776958,a2021-05-18t12:23:18b_c9660679917d_e9254535117...
